In [8]:
import sys
print(sys.executable)

c:\Users\H.P\AppData\Local\Programs\Python\Python311\python.exe


In [2]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from IPython.display import display, Markdown
import ipywidgets as widgets
from ipywidgets import interact





In [3]:





jobs = pd.read_csv("Jobs.csv")
skills = pd.read_csv("Skills.csv")
companies = pd.read_csv("CompanyHiring.csv")

# Clean GrowthRate
jobs['GrowthRate'] = jobs['GrowthRate'].str.rstrip('%').astype(float)
skills['GrowthRate'] = skills['GrowthRate'].str.rstrip('%').astype(float)

# Split skills in companies
companies = companies.assign(TopSkills=companies['TopSkills'].str.split(', ')).explode('TopSkills')




In [4]:





industry_filter = widgets.Dropdown(
    options=['All'] + sorted(companies['Industry'].dropna().unique().tolist()),
    value='All',
    description='Industry:'
)

location_filter = widgets.Dropdown(
    options=['All'] + sorted(jobs['Location'].dropna().unique().tolist()),
    value='All',
    description='Location:'
)

skill_filter = widgets.Dropdown(
    options=['All'] + sorted(skills['Category'].dropna().unique().tolist()),
    value='All',
    description='Skill Cat:'
)




In [5]:



def update_dashboard(industry, location, skill_cat):
    # Filter companies
    comp_df = companies.copy()
    if industry != 'All':
        comp_df = comp_df[comp_df['Industry'] == industry]
    
    # Filter jobs
    jobs_df = jobs.copy()
    if location != 'All':
        jobs_df = jobs_df[jobs_df['Location'] == location]
    
    # Filter skills
    skills_df = skills.copy()
    if skill_cat != 'All':
        skills_df = skills_df[skills_df['Category'] == skill_cat]
    
    # --- KPIs ---
    display(Markdown("## 🚀 Career Insights Dashboard (Interactive)"))
    if not jobs_df.empty:
        top_job = jobs_df.loc[jobs_df['DemandScore'].idxmax()]
        display(Markdown(f"*Top Demand Job:* {top_job['JobTitle']} ({top_job['DemandScore']})"))
    if not skills_df.empty:
        top_skill = skills_df.loc[skills_df['GrowthRate'].idxmax()]
        display(Markdown(f"*Highest Growth Skill:* {top_skill['SkillName']} ({top_skill['GrowthRate']}% growth)"))
    if not comp_df.empty:
        top_company = comp_df.groupby('CompanyName')['JobsHired'].sum().idxmax()
        display(Markdown(f"*Top Hiring Company:* {top_company}"))
    
    # --- Jobs Analysis ---
    if not jobs_df.empty:
        fig = px.bar(jobs_df.sort_values('DemandScore', ascending=False),
                     x='DemandScore', y='JobTitle', color='DemandScore',
                     title="Top Jobs by Demand Score")
        fig.show()
        
        fig = px.scatter(jobs_df, x="DemandScore", y="GrowthRate", color="Location",
                         size="DemandScore", hover_data=["JobTitle"],
                         title="Job Demand vs Growth Rate by Location")
        fig.show()
    
    # --- Skills Analysis ---
    if not skills_df.empty:
        fig = px.treemap(skills_df, path=['Category','SkillName'], values='DemandScore',
                         color='GrowthRate', color_continuous_scale='Blues',
                         title="Skills Demand & Growth")
        fig.show()
        
        fig = px.scatter(skills_df, x="DemandScore", y="GrowthRate", text="SkillName", size="DemandScore",
                         color="Category", title="Skills: Demand vs Growth Rate")
        fig.update_traces(textposition='top center')
        fig.show()
    
    # --- Company Analysis ---
    if not comp_df.empty:
        comp_summary = comp_df.groupby('CompanyName')['JobsHired'].sum().reset_index()
        fig = px.bar(comp_summary.sort_values('JobsHired', ascending=False),
                     x='CompanyName', y='JobsHired', color='JobsHired',
                     title="Top Hiring Companies")
        fig.show()
        
        fig = px.scatter(comp_df, x="CompanyName", y="TopSkills", size="JobsHired", color="Industry",
                         hover_data=['JobsHired'], title="Companies & Skills Hiring")
        fig.show()



In [6]:



interact(update_dashboard, industry=industry_filter, location=location_filter, skill_cat=skill_filter)

interactive(children=(Dropdown(description='Industry:', options=('All', 'Consulting', 'Healthcare', 'IT'), val…

<function __main__.update_dashboard(industry, location, skill_cat)>